<a href="https://colab.research.google.com/github/matwirabc/binaproject/blob/aggregate/bina_aggregate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Einführung
Im folgenden Abschnitt werden die Ausganglage beschrieben, sowie die Zielsetzung erörter.

##1.1 Vorgehen
Die Arbeit ist nach dem "Five-Step Approach" nach CPA (Chartered Professional Accountants) gegliedert. Die fünf Schritte sind:


1.   Ziel(e) setzen
2.   Datenerhebung
3.   Analyse
4.   Aufarbeiten
5.   Entscheiden


##1.2 Ausgangslage
Die zuvor bearbeiteten Datensätze stammten aus unterschiedlichen Quellen und standen bislang in keinem Zusammenhang. In diesem Abschnitt sollen die bereinigten Datensätze nun miteinander in Beziehung gesetzt werden. Folgende Datensätze werden dafür genutzt:


*   Fahrzeuge
*   Flugzeuge
*   Bahn
*   Bevölkerung Schweiz


Der Datensatz "2.4 Bevölkerung Schweiz" soll dabei als verbindendes Element genutz werden.

# Schritt 1: Zielsetzung
Folgende Fragen sollen in der Arbeit bearbeitet und beantwortet werden.

**Hauptfrage:**
Wie haben sich die Verkehrsmittel im Verhältnis zum Bevölkerungswachstum in der Schweiz entwickelt – sowohl relativ als auch absolut?

**Unterfrage 1**
Welches Verkehrsmittel verzeichnete im Untersuchungszeitraum das stärkste Wachstum (relativ und absolut)?

**Unterfrage 2**
Welches Verkehrsmittel erlitt während der Corona-Pandemie den stärksten Rückgang (relativ und absolut)?

# Schritt 2: Datenerhebung
Für die Anaylse der Entwicklung und Trends wurden die Datensätze aus den anderen Bereichen dieser Arbeit verwendet. Die Datensätze sind im GIT-Repository abgelegt.

##2.1 Laden und importieren der Libraries
Die Verarbeitung und Darstellung der Daten erfolgt mit den untenstehenden Libarys, welche in einem ersten Schritt importiert werden.


In [40]:
#Load necessary packages/libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import matplotlib.ticker as mticker
import plotly.express as px
import numpy as np
import plotly.graph_objs as go
import warnings
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress
from plotly.subplots import make_subplots
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from google.colab import drive
drive.mount('/content/drive')
!pip install dash
!pip install mplcursors



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##2.2 Laden der Datensets
Die Datensets wurden in verschiedenen Formaten (.csv und .xlsx) zur Verfügung gestellt.

###2.2.1 Laden der .csv Daten

In [150]:
# Load Data
df_car = pd.read_csv("/content/drive/MyDrive/Dataset_BINA/Cars.csv",
                sep=';',
                encoding='latin-1',
                quotechar='"')
df_plane = pd.read_csv("/content/drive/MyDrive/Dataset_BINA/Planes.csv",
                sep=',',
                parse_dates=['firstseen', 'lastseen'])

### 2.2.2 Laden der .xlsx Daten
Da die Excel aus mehreren "Sheets" bestehen, soll nicht nur das erste sondern alle Sheets geladen werden.

In [49]:
# Load all sheets  as dictionary by function sheet_name = None
df_population = pd.read_excel("/content/drive/MyDrive/Dataset_BINA/Population.xlsx",
                              sheet_name = None,
                              header = 5)

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

###2.2.3 Prüfen der importierten Daten
Um zu prüfen, ob die Daten korrekt geladen wurden wird jeder Datensatz mittels df_Datensatz.head() aufgerufen. Dadurch werden die ersten fünf Zeilen des Datensatzes dargestellt.

In [151]:
df_car.head()

,Gemeinde,Fahrzeuggruppe,Treibstoff,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Gemeindename,PLZ,Kantonskuerzel
0,Aeugst am Albis,Personenwagen,Benzin,881,856,845,822,815,816,809,804,789,778,774,724,Aeugst am Albis,8914.0,ZH
1,Aeugst am Albis,Personenwagen,Benzin,881,856,845,822,815,816,809,804,789,778,774,724,Aeugst am Albis,8914.0,ZH
2,Aeugst am Albis,Personenwagen,Diesel,250,276,288,306,316,318,326,329,321,311,292,287,Aeugst am Albis,8914.0,ZH
3,Aeugst am Albis,Personenwagen,Diesel,250,276,288,306,316,318,326,329,321,311,292,287,Aeugst am Albis,8914.0,ZH
4,Aeugst am Albis,Personenwagen,Benzin-elektrisch: Normal-Hybrid,11,13,13,18,16,20,22,30,43,49,57,74,Aeugst am Albis,8914.0,ZH


In [114]:
df_plane.head()

,icao24,firstseen,departure_airport,lastseen,arrival_airport,callsign,query_airport,flight_type,network_duration_minutes,airline_code,locality
0,3c942d,2016-03-03 09:17:57+00:00,LSZH,2016-03-03 09:27:59+00:00,EDNY,DBEAM,LSZH,Departure,10.033333,DBE,International
1,4b1613,2016-03-03 20:14:39+00:00,LSZH,2016-03-03 21:31:36+00:00,EGLW,SWR24A,LSZH,Departure,76.950000,SWR,International
2,4407da,2016-03-03 18:10:16+00:00,LSZH,2016-03-03 19:00:56+00:00,LOAV,NLY8611,LSZH,Departure,50.666667,NLY,International
3,400aff,2016-03-03 19:14:16+00:00,LSZH,2016-03-03 20:36:55+00:00,EGLW,BAW719,LSZH,Departure,82.650000,BAW,International
4,4407f3,2016-03-03 20:38:48+00:00,LSZH,2016-03-03 21:30:29+00:00,LOAN,NLY115E,LSZH,Departure,51.683333,NLY,International


In [46]:
#Show first five rows of each sheet
for df_population, df in df_population.items():
    print(f"\n--- Sheet: {df_population} ---")
    display(df.head())


--- Sheet: 2023 ---


,Schweiz,8962258,1788440,5443532,1730286,4451532,4510726,6544970,2417288,4127991,3615921,403261,797858,733,11722,4018,5643707,1962454,1356097
0,Genferseeregion,1736124.0,365836.0,1066594.0,303694.0,851611.0,884513.0,1140993.0,595131.0,842801.0,654392.0,72185.0,162605.0,235.0,2750.0,926.0,1200494.0,370335.0,165295.0
1,Waadt,845870.0,184231.0,519663.0,141976.0,415697.0,430173.0,561410.0,284460.0,418325.0,315182.0,33595.0,76804.0,90.0,1292.0,444.0,521241.0,236125.0,88504.0
2,Wallis,365844.0,70577.0,219937.0,75330.0,182413.0,183431.0,273857.0,91987.0,163137.0,149547.0,18664.0,33968.0,44.0,310.0,114.0,201600.0,87453.0,76791.0
3,Genf,524410.0,111028.0,326994.0,86388.0,253501.0,270909.0,305726.0,218684.0,261339.0,189663.0,19926.0,51833.0,101.0,1148.0,368.0,477653.0,46757.0,0.0
4,Espace Mittelland,1944753.0,385379.0,1154922.0,404452.0,961157.0,983596.0,1538772.0,405981.0,881538.0,784279.0,95409.0,180496.0,145.0,2007.0,737.0,973102.0,519662.0,451989.0



--- Sheet: 2022 ---


,Schweiz,8815385,1755213,5368549,1691623,4379953,4435432,6519362,2296023,4024051,3589295,402798,780357,647,14293,3682,5547579,1932177,1335629
0,Genferseeregion,1701827.0,358186.0,1046653.0,296988.0,835051.0,866776.0,1130600.0,571227.0,816952.0,649510.0,72194.0,158686.0,228.0,3360.0,849.0,1176727.0,363754.0,161346.0
1,Waadt,830431.0,180828.0,510764.0,138839.0,408209.0,422222.0,554772.0,275659.0,405980.0,313352.0,33611.0,75333.0,83.0,1646.0,405.0,511867.0,232420.0,86144.0
2,Wallis,357282.0,68941.0,215062.0,73279.0,177835.0,179447.0,272671.0,84611.0,157310.0,148115.0,18653.0,32679.0,38.0,373.0,100.0,196906.0,85174.0,75202.0
3,Genf,514114.0,108417.0,320827.0,84870.0,249007.0,265107.0,303157.0,210957.0,253662.0,188043.0,19930.0,50674.0,107.0,1341.0,344.0,467954.0,46160.0,0.0
4,Espace Mittelland,1918746.0,379054.0,1144199.0,395493.0,948765.0,969981.0,1536189.0,382557.0,862111.0,781413.0,95513.0,176418.0,121.0,2438.0,678.0,958079.0,513602.0,447065.0



--- Sheet: 2021 ---


...

,Schweiz,7255653,1642753,4481755,1131145,25.238885,3544349,3711304,5808100,1447553,4550410,1532621,1172622
0,Genferseeregion,1317017.0,301841.0,815363.0,199813.0,24.506018,635646.0,681371.0,948786.0,368231.0,919119.0,260732.0,137166.0
1,Waadt,624980.0,145885.0,381556.0,97539.0,25.563482,301829.0,323151.0,457604.0,167376.0,397405.0,162731.0,64844.0
2,Wallis,278419.0,65572.0,170997.0,41850.0,24.474114,136648.0,141771.0,232646.0,45773.0,142247.0,63850.0,72322.0
3,Genf,413618.0,90384.0,262810.0,60424.0,22.991515,197169.0,216449.0,258536.0,155082.0,379467.0,34151.0,0.0
4,Espace Mittelland,1667070.0,378031.0,1013610.0,275429.0,27.173074,812989.0,854081.0,1430578.0,236492.0,832194.0,435369.0,399507.0



--- Sheet: 2000 ---


,Schweiz,7204055,1664351,4430518,1109186,25.035131,3519698,3684357,5779685,1424370,4516343,1516853,1170859
0,Genferseeregion,1305284.0,302668.0,806348.0,196268.0,24.340359,629459.0,675825.0,940472.0,364812.0,909737.0,257982.0,137565.0
1,Waadt,620294.0,145695.0,377908.0,96691.0,25.585857,299184.0,321110.0,455127.0,165167.0,394696.0,160418.0,65180.0
2,Wallis,276170.0,67107.0,168911.0,40152.0,23.771098,135543.0,140627.0,230520.0,45650.0,139994.0,63791.0,72385.0
3,Genf,408820.0,89866.0,259529.0,59425.0,22.897248,194732.0,214088.0,254825.0,153995.0,375047.0,33773.0,0.0
4,Espace Mittelland,1658575.0,384790.0,1002872.0,270913.0,27.013717,809708.0,848867.0,1426966.0,231609.0,828382.0,431814.0,398379.0



--- Sheet: 1999 ---


,Schweiz,7164444,1663861,4406320,1094263,24.833943,3500708,3663736,5757814,1406630,4490375,1502698,1171371
0,Genferseeregion,1294974.0,300761.0,800613.0,193600.0,24.181471,624640.0,670334.0,934144.0,360830.0,901442.0,255965.0,137567.0
1,Waadt,616275.0,144728.0,375556.0,95991.0,25.559703,297405.0,318870.0,453666.0,162609.0,392439.0,158806.0,65030.0
2,Wallis,275632.0,67790.0,168576.0,39266.0,23.292758,135321.0,140311.0,229775.0,45857.0,139496.0,63599.0,72537.0
3,Genf,403067.0,88243.0,256481.0,58343.0,22.747494,191914.0,211153.0,250703.0,152364.0,369507.0,33560.0,0.0
4,Espace Mittelland,1656109.0,386239.0,1001186.0,268684.0,26.836572,809913.0,846196.0,1427796.0,228313.0,828335.0,428771.0,399003.0


##2.3 Bereinigen der Datensets
Es werden nicht alle Daten in den Datensets benötigt. Deshalb werden diese im folgenden Bereinigt.

### 2.3.1 Fahrzeuge
Der zur Verfügung gestellt Datensatz zu den Fahrzeugen ist nach Gemeinden geordnet. Diese detailierte Aufschlüsselung ist für die Analyse nicht notwendig. Es soll die Entwicklung pro Fahrzeugtyp über die Jahre erfasst werden. Die Fahrzeugtypen sind im Datensatz nach Antriebs-/Treibstoff Art klassifiziert. Diese sind:

*   Benzin
*   Diesel
*   Benzin-elektrisch: Normal Hybrid
*   Benzin-elektrisch: Plugin Hybrid
*   Diesel-elektrisch: Normal Hybrid
*   Diesel-elektrisch: Plugin Hybrid
*   Elektrisch
*   Wasserstoff
*   Ohne Motor
*   Andere

Da Fahrzeuge "Ohne Motor" per Definition keine Fahrzeuge sind und zudem für die Erhebung irrelevant sind, werden diese aus dem Datensatz entfernt.

In [161]:
#Check name of each row
print(df_car.columns.tolist())

#Delete not needed rows
df_car_adj = df_car.drop(columns=['Gemeinde', 'Fahrzeuggruppe', 'Gemeindename', 'PLZ',])

#Check name and values of first fife rows after adjustment
df_car_adj.head()

# Jahres-Spalten automatisch erkennen (Spaltennamen, die nur aus Ziffern bestehen)
year_columns = [col for col in df_car_adj.columns if col.isdigit()]

# Daten transformieren: Wide → Long Format
df_melted = df_car_adj.melt(
    id_vars=['Treibstoff'],
    value_vars=year_columns,
    var_name='Jahr',
    value_name='Anzahl'
)

# Group and create sum by Treibstoff und Jahr
df_car_adj = df_melted.groupby(['Treibstoff', 'Jahr'])['Anzahl'].sum().reset_index()

# Create pivot table
df_car_adj = df_car_adj.pivot(
    index='Treibstoff',
    columns='Jahr',
    values='Anzahl'
).fillna(0).astype(int).reset_index()

#Remove row "Ohne Motor"
df_car_adj = df_car_adj[df_car_adj['Treibstoff'].str.lower() != 'ohne motor']

# Show results
print(df_car_adj)

['Gemeinde', 'Fahrzeuggruppe', 'Treibstoff', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', 'Gemeindename', 'PLZ', 'Kantonskuerzel']
Jahr                         Treibstoff      2013      2014      2015  \
0                               Anderer     27156     25189     24712   
1                                Benzin  18461927  18165476  17926933   
2      Benzin-elektrisch: Normal-Hybrid    204170    235404    265373   
3     Benzin-elektrisch: Plug-in-Hybrid       709      2721     12087   
4                                Diesel   6198043   6687059   7178578   
5      Diesel-elektrisch: Normal-Hybrid      9391     12516     16852   
6     Diesel-elektrisch: Plug-in-Hybrid       916      1597      2438   
7                            Elektrisch     16634     27885     45947   
8              Gas (mono- und bivalent)     59562     64069     68133   
10                          Wasserstoff        24        24        24   

Jahr      2016

### 2.3.2 Flugzeuge

Für die Analyse der Flugnutzung durch die Schweizer Wohnbevölkerung ist es sinnvoll, sich primär auf ausgehende Flüge zu konzentrieren. Diese spiegeln die tatsächliche Mobilität der in der Schweiz lebenden Personen wider – sei es im Rahmen von Urlaubs-, Geschäfts- oder Auswanderungsreisen. Eingehende Flüge aus dem Ausland hingegen betreffen häufig ausländische Touristen, Transitreisende oder Rückkehrer, die mit dem Mobilitätsverhalten der Schweizer Bevölkerung nur bedingt in Zusammenhang stehen. Um valide Aussagen über Veränderungen im Reiseverhalten zu treffen, sollte die Betrachtung daher auf Departures von Schweizer Flughäfen beschränkt werden. Die Flüge werden nach Jahr geclustert und es wird dabei noch folgende Arten unterschieden:

*   International
*   Domestic
*   Total (Summe aus Internationa und Domestic)





In [133]:
#Check name of each row
print(df_plane.columns.tolist())

#Read year from row lastseen, and create new row "year"
df_plane['year'] = df_plane['lastseen'].astype(str).str[:4]

#Delete not needed rows
df_plane_adj = df_plane.drop(columns=['icao24', 'firstseen', 'departure_airport', 'lastseen', 'callsign', 'query_airport', 'network_duration_minutes', 'airline_code',])

#Check name and values of first fife rows after adjustment
df_plane_adj.head()

#Create Sum of
# Total number of flights per year
df_flights_per_year = df_plane_adj.groupby('year').size().reset_index(name='total_flights')

# Count international flight
df_international_flights = df_plane_adj[df_plane_adj['locality'].str.lower() == 'international']
df_flights_international = df_international_flights.groupby('year').size().reset_index(name='df_international_flights')

# Count domestic flights
df_domestic_flights = df_plane_adj[df_plane_adj['locality'].str.lower() == 'domestic']
df_flights_domestic = df_domestic_flights.groupby('year').size().reset_index(name='df_domestic_flights')

# 4. Merge (Total is sum of international and domestic flights)
df_plane_adj = df_flights_per_year \
    .merge(df_flights_international, on='year', how='left') \
    .merge(df_flights_domestic, on='year', how='left') \
    .fillna(0)

# Show results
print(df_plane_adj)

['icao24', 'firstseen', 'departure_airport', 'lastseen', 'arrival_airport', 'callsign', 'query_airport', 'flight_type', 'network_duration_minutes', 'airline_code', 'locality', 'year']
   year  total_flights  df_international_flights  df_domestic_flights
0  2016         132969                    119095                13874
1  2017         276494                    261580                14914
2  2018         318017                    298727                19290
3  2019         346812                    325983                20829
4  2020         168910                    156065                12845
5  2021         196349                    181757                14592
6  2022         331563                    312168                19395
7  2023         369343                    345730                23613
8  2024         395165                    368497                26668


### 2.3.3 Bevölkerung
Bei der Bevölkerung ist das Wachstum der Bevölkerung in der Schweiz relevant.

In [62]:
#load with "header = None", for reverence to fixed table reference b6
df_population = pd.read_excel("/content/drive/MyDrive/Dataset_BINA/Population.xlsx",
                              sheet_name = None,
                              header = None)

# Extract cell B6 from each sheet (corresponds to row 6, column 2 → [5, 1])
result = {}

for sheet_name, df in df_population.items():
    try:
        result[sheet_name] = df.iloc[5, 1]
    except Exception as e:
        result[sheet_name] = None

# Show results
df_population_adj = pd.DataFrame(result.items(), columns=["Sheetname", "Value_B6"])
df_population_adj

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""C

,Sheetname,Value_B6
0,2023,8962258
1,2022,8815385
2,2021,8738791
3,2020,8670300
4,2019,8606033
5,2018,8544527
6,2017,8484130
7,2016,8419550
8,2015,8327126
9,2014,8237666


#3. Analyse


# 4. Aufarbeiten

# 5. Entscheiden
